<div class="alert alert-block alert-info">
<h3>Student Information</h3> Please provide information about yourself.<br>
<b>Name</b>: Vinh Huynh<br>
<b>NetID</b>: vbh11<br>
<b>Recitation (01/02)</b>: 2<br>
<b>Notes to Grader</b> (optional):<br>
<br><br>
<b>IMPORTANT</b>
Your work will not be graded withour your initials below<br>
I certify that this lab represents my own work and I have read the RU academic intergrity policies at<br>
<a href="https://www.cs.rutgers.edu/academic-integrity/introduction">https://www.cs.rutgers.edu/academic-integrity/introduction </a><br>
<b>Initials</b>: vh     


<h3>Grader Notes</h3>
<b>Your Grade<b>:<br>
<b>Grader Initials</b>:<br>
<b>Grader Comments</b> (optional):<br>
</div>

# Lab 7: Linear Models, Feature Engineering and Logistic Regression

** This lab is due Monday April 22, 2019 at 11:59pm (graded on accuracy and completeness) **

In this lab we will work through the process of:
1. implementing a linear model, defining loss functions, 
2. feature engineering,
3. minimizing loss functions using numeric libraries 
4. how to implement logistic regression model for a binary classification problem

We will use the toy tip calculation dataset from sns.

## Set up

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
np.random.seed(42)
plt.style.use('fivethirtyeight')
sns.set()
sns.set_context("talk")
%matplotlib inline

## Load the Tips Dataset

To begin with, we load the tips dataset from the `seaborn` library.  The tips data contains records of tips, total bill, and information about the person who paid the bill.

In [2]:
data = sns.load_dataset("tips")
print("Number of Records:", len(data))
data.head()

Number of Records: 244


,total_bill,tip,sex,smoker,day,time,size
0,16.99,1.01,Female,No,Sun,Dinner,2
1,10.34,1.66,Male,No,Sun,Dinner,3
2,21.01,3.50,Male,No,Sun,Dinner,3
3,23.68,3.31,Male,No,Sun,Dinner,2
4,24.59,3.61,Female,No,Sun,Dinner,4


# Part 1: Defining the Model and Feature Engineering

In lab 6, we defined a simple linear model with only one parameter. Now let's make a more complicated model that utilizes other features in a dataset. Let our prediction for tip be a combination of the following features:

$$
\text{Tip} = \theta_1 * \text{total_bill} + \theta_2 * \text{sex} + \theta_3 * \text{smoker} + \theta_4 * \text{day} + \theta_5 * \text{time} + \theta_6 * \text{size}
$$

Notice that some of these features are not numbers! But our linear model will need to predict a numerical value. 
### Task 1.1 Split the data into Y(tip) and X(all others)

In [90]:
## BEGIN SOLUTION
#X are the features
tips = data['tip']
X = data.drop('tip',axis=1)
## END SOLUTION

## Task 1.2: Feature Engineering
First, let's convert everything to numerical values. A straightforward approach is to map some of these non-numerical features into numerical ones. For example, we can treat the day as a value from 1-7. However, one of the disadvantages in directly translating to a numeric value is that we unintentially assign certain features disproportionate weight. Consider assigning Sunday to the numeric value of 7. Monday is assigned to 1 and thus Sunday has 7 times the influence of Monday in our linear model which can lower the accuracy of our model.

solution: one-hot encoding! 

As discussed in lecture 9.2, one-hot encoding will produce a binary vector indicating the non-numeric feature. Sunday would be encoded as a [0 0 0 0 0 0 1]. This assigns a more even weight across each category in non-numeric features. Complete the code below to one-hot encode our dataset.

In [5]:
def one_hot_encode(data):
    """
    Return the one-hot encoded dataframe of our input, data. 
    Hint: check pd.get_dummies
    """
    ### BEGIN SOLUTION
    hotEncode = pd.get_dummies(data)
    return hotEncode
    #return hotEncodeweek
    ### END SOLUTION
    
one_hot_encode(data).head()

,total_bill,tip,size,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,16.99,1.01,2,0,1,0,1,0,0,0,1,0,1
1,10.34,1.66,3,1,0,0,1,0,0,0,1,0,1
2,21.01,3.50,3,1,0,0,1,0,0,0,1,0,1
3,23.68,3.31,2,1,0,0,1,0,0,0,1,0,1
4,24.59,3.61,4,0,1,0,1,0,0,0,1,0,1


In [8]:
one_hot_X = one_hot_encode(X)
one_hot_X.head()

,total_bill,size,sex_Male,sex_Female,smoker_Yes,smoker_No,day_Thur,day_Fri,day_Sat,day_Sun,time_Lunch,time_Dinner
0,16.99,2,0,1,0,1,0,0,0,1,0,1
1,10.34,3,1,0,0,1,0,0,0,1,0,1
2,21.01,3,1,0,0,1,0,0,0,1,0,1
3,23.68,2,1,0,0,1,0,0,0,1,0,1
4,24.59,4,0,1,0,1,0,0,0,1,0,1


## Task 1.2: Defining the Model
Now that all of our data is numeric, let's define our model function. Note that X and thetas are matrices now. Use matrix products to compute the model.

In [91]:
def linear_model(thetas, X):
    """
    Return the linear combination of thetas and features as defined above.
    """
    ### BEGIN SOLUTION
    #print(thetas.shape)
    #print(X.shape)
    return np.matmul(X, thetas)
    ### END SOLUTION

#linear_model(np.arange(1,5), np.arange(1,5))
#linear_model(np.arange(1,13), one_hot_X)

In [8]:
assert linear_model(np.arange(1,5), np.arange(1,5)) == 30

# Part 2: Fitting the Model: Numeric Methods
Recall in the previous lab we defined multiple loss functions and found optimal theta using the scipy.minimize function. Adapt the loss functions and optimization code from the previous lab (provided below) to work with your new linear model.

In [96]:
from scipy.optimize import minimize

def abserror(y, y_hat):
    return np.abs(y-y_hat)

def sqerror(y, y_hat):
    return (y - y_hat)**2

def minimize_average_loss(loss_function, model, x, y):
    """
    loss_function: either the squared or absolute loss functions from above.
    model: the model (as defined above)
    x: the x values (one-hot encoded data)
    y: the y values (tip amounts)
    return the estimate for each theta as a vector
    
    Note we will ignore failed convergence for this lab ... 
    """
    
    ## Notes on the following function call which you need to finish:
    # 
    # 0. the ... should be replaced with the average loss evaluated on 
    #       the data x, y using the model and appropriate loss function
    # 1. x0 are the initial values for THETA.  Yes, this is confusing
    #       but optimization people abslike x to be the thing they are 
    #       optimizing.
    # 2. We extract the 'x' entry in the dictionary which corresponds
    #       to the value of thetas at the optimum
    ### BEGIN SOLUTION
    #x0 = np.random.rand(244,)
    #res = minimize(loss_function, x0)
    
    return minimize(lambda theta: loss_function(model(theta, x), y).mean(), x0=np.zeros(x.shape[1]))['x']
    ### END SOLUTION
    ## return minimize(lambda theta: ..., x0=...)

minimize_average_loss(sqerror, linear_model, one_hot_X, tips)
#Solution output:
#array([0.17599118, 0.09448702, 0.01519968, 0.1774608 , 0.05600943,
#       0.15199342, 0.18411066, 0.2165521 , 0.15712758, 0.24353557,
#       0.23440138, 0.16626163])

array([0.09448701, 0.17599126, 0.18411065, 0.21655212, 0.15712761,
       0.24353555, 0.01519939, 0.17746096, 0.0560094 , 0.15199333,
       0.23440157, 0.16626158])

# Part 3: Fitting the Model: Analytic Methods
Let's also fit our model analytically, for the l2 loss function. In this question we will derive an analytical solution, fit our model and compare our results with our numerical optimization results.

## Task 3.1: Least Squares Solution
Recall that if we're fitting a linear model with the l2 loss function, we are performing least squares! Remember, we are solving the following optimization problem for least squares:

$$\min_{\theta} ||X\theta - y||^2$$

Let's begin by deriving the analytic solution to least squares. Write your answer in LaTeX in the cell below. Assume X is full column rank. You are given step 1 to get things started. You need to complete the steps 2-4 where step 4 shows the value of theta that optimizes given X and y.

### BEGIN SOLUTION
Take the gradient and set theta equal to 0. 

$$2(X\theta - y) = 0$$
$$ 2x\theta = 2y$$
$$ x\theta = y$$
$$ \theta = x^{-1}y$$
### END SOLUTION

## Task 3.2: Solving for Theta
Using your formuala above, let us find the analytic solution for $\theta$. That is, the optimal numerical thetas for our tips dataset. Fill out the function below. Make sure you use the float type in your calculations using .astype(float) and use the np.linalg.inv function.

In [81]:
def get_analytical(x, y):
    """
    x: our one-hot encoded dataset
    y: tip amounts
    """
    ### BEGIN SOLUTION
    return np.dot(np.linalg.pinv(x), y)
    ### END SOLUTION
    
analytical_thetas = get_analytical(one_hot_X.astype(float), tips.astype(float))
#print(analytical_thetas)
#print(linear_model(analytical_thetas, one_hot_X))
print(sqerror(linear_model(analytical_thetas, one_hot_X),tips).mean())

1.010353561225785


In [97]:
analytical_thetas = get_analytical(one_hot_X.astype(float), tips.astype(float))
print("Our analytical loss is: ", sqerror(linear_model(analytical_thetas, one_hot_X),tips).mean())
print("Our numerical loss is: ", sqerror(linear_model(minimize_average_loss(sqerror, linear_model, one_hot_X, tips), one_hot_X), tips).mean())
#Our analytical loss is:  59.733414754098355
#Our numerical loss is:  1.0103535612368402

Our analytical loss is:  1.010353561225785
Our numerical loss is:  1.010353561236597


In [11]:
assert np.isclose(sqerror(linear_model(analytical_thetas, one_hot_X),tips).mean(), 59.733414754098362)

## Task 3.3: Weird Results?
Our analytical loss is surprisingly much worse than our numerical loss. Why is this? 

Hint: https://stackoverflow.com/questions/31256252/why-does-numpy-linalg-solve-offer-more-precise-matrix-inversions-than-numpy-li

### BEGIN SOLUTION



### END SOLUTION

# Part 4 - Logistic Regression
In this part we will implement a logistic regression model with the tip data set. We will use Male and Female as the binary classification. We will use the LogisiticRegression model from sklearn to find a prediction model.

## task 4.1
** create the X (one-hot features) and y (sex) **

In [40]:
## BEGIN SOLTUION
dataTwo = sns.load_dataset("tips")
X = dataTwo.drop(['sex'],axis=1)
X = one_hot_encode(X)
y = dataTwo['sex']
## END SOLUTION
print(X.head())
print(y.head())


   total_bill   tip  size  smoker_Yes  smoker_No  day_Thur  day_Fri  day_Sat  \
0       16.99  1.01     2           0          1         0        0        0   
1       10.34  1.66     3           0          1         0        0        0   
2       21.01  3.50     3           0          1         0        0        0   
3       23.68  3.31     2           0          1         0        0        0   
4       24.59  3.61     4           0          1         0        0        0   

   day_Sun  time_Lunch  time_Dinner  
0        1           0            1  
1        1           0            1  
2        1           0            1  
3        1           0            1  
4        1           0            1  
0    Female
1      Male
2      Male
3      Male
4    Female
Name: sex, dtype: category
Categories (2, object): [Male, Female]


## task 4.2 
Split the data set to training and test using sklearn train_test_split

In [41]:
from sklearn.model_selection import train_test_split
## BEGIN SOLUTION
X_train, X_test, y_train, y_test = train_test_split(X, y)
## END SOLUTION

print (X_train.head(), X_test.head(), y_train.head(), y_test.head())

     total_bill   tip  size  smoker_Yes  smoker_No  day_Thur  day_Fri  \
156       48.17  5.00     6           0          1         0        0   
71        17.07  3.00     3           0          1         0        0   
231       15.69  3.00     3           1          0         0        0   
168       10.59  1.61     2           1          0         0        0   
138       16.00  2.00     2           1          0         1        0   

     day_Sat  day_Sun  time_Lunch  time_Dinner  
156        0        1           0            1  
71         1        0           0            1  
231        1        0           0            1  
168        1        0           0            1  
138        0        0           1            0        total_bill   tip  size  smoker_Yes  smoker_No  day_Thur  day_Fri  \
163       13.81  2.00     2           0          1         0        0   
107       25.21  4.29     2           1          0         0        0   
80        19.44  3.00     2           1         

## task 4.3 logistic regression model
Fit a logisitic regression model using LogisticRegression from sklearn

In [45]:
from sklearn.linear_model import LogisticRegression
## BEGIN SOLUTION
logmodel = LogisticRegression().fit(X_train, y_train)
logmodel
## END SOLUTION

C:\Users\Vinh Huynh\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

## task 4.4 Predicting
Now we can use the logmodel from previous part to predict on the test data. Read documentation to see how. It should be just a one line of code.

In [49]:
## BEGIN SOLUTION
predictions = logmodel.predict(X_test)
predictions
## END SOLUTION

array(['Male', 'Male', 'Female', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Female', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Female', 'Male', 'Female', 'Male',
       'Female', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Female', 'Female',
       'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male', 'Male',
       'Female', 'Male', 'Male', 'Male', 'Female', 'Male', 'Male', 'Male',
       'Male', 'Female', 'Female', 'Male', 'Female'], dtype=object)

## task 4.5 reporting
use classification_report from sklearn.metrics to report the accuracy of the model.

In [52]:
from sklearn.metrics import classification_report
## BEGIN SOLUTION
targetNames = ["Male", "Female"]
print(classification_report(predictions, y_test, target_names=targetNames))
## END SOLUTION

              precision    recall  f1-score   support

        Male       0.29      0.42      0.34        12
      Female       0.84      0.76      0.80        49

   micro avg       0.69      0.69      0.69        61
   macro avg       0.57      0.59      0.57        61
weighted avg       0.73      0.69      0.71        61



### Feedback
Please provide feedback on this lab.
* how would you rate this lab (from 1-10, 10-highest) : 7
* how can we improve his lab? : None

<div class="alert alert-block alert-info">
<h2>Submission Instructions</h2> 
<b> File Name:</b> Please name the file as your_section_your_netID_lab7.jpynb<br>
<b> Submit To: </b> Canvas &rarr; Assignments &rarr; lab7 <br>
<b>Warning:</b> Failure to follow directions may result in loss points.<br>
</div>

Developed by A.D. Gunawardena. Credits: Josh Hug, Berkeley Data Science Lab